In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
from utils.datasets import all_datasets
import matplotlib.pyplot as plt 
import numpy as np

from utils.cnn_duq import CNN_DUQ
from utils.cnn_duq import SoftmaxModel as CNN

from utils.resnet import ResNet
from utils.resnet_duq import ResNet_DUQ


ds3 = all_datasets["CIFAR10"]()
input_size, num_classes, _ , test_dataset_CIFAR = ds3
ds4 = all_datasets["SVHN"]()
_,_,_, test_dataset_SVHN = ds4


mod='DUQ' #['DUQ','DE']

if mod=='DUQ':
  model_CIFAR = ResNet_DUQ(input_size, num_classes, 512 ,512 ,0.1 ,0.999).cuda() 
  model_CIFAR.load_state_dict(torch.load("/content/gdrive/My Drive/Colab Notebooks/DUQ_CIFAR_75.pt"))
else:
  ensemble = [
            ResNet(input_size, num_classes).cuda() for _ in range(5)
        ]
  model_CIFAR = torch.nn.ModuleList(ensemble);
  model_CIFAR.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/CIFAR10_5_ensemble.pt'))
model_CIFAR.eval()

b=50
r=len(test_dataset_CIFAR)
ls=[]
m=test_dataset_CIFAR[0][0].max()


for i in range(int(r/b)+1):
    data=[]
    for j in range(0,b):
        c=i*b+j
        if(c>=r):
          break
        data.append(test_dataset_CIFAR[c][0])
    if(len(data)==0):
      break
    data=torch.stack(data)
    ls.append(data)


kernel_distace_SVHN = np.zeros((r))
for i in range(int(r/b)+1):
  data=[]
  for j in range(0,b):
      c=i*b+j
      if(c>=r):
        break
      data.append(test_dataset_SVHN[c][0])
  if(len(data)==0):
    break
  data=torch.stack(data)

  if mod=='DUQ':
    res=model_CIFAR(data.cuda())[1].max(1)[0].detach()
  else:
    predictions = torch.stack([model(data.cuda()) for model in model_CIFAR])
    mean_prediction = torch.mean(predictions.exp(), dim=0)
    res=torch.sum(mean_prediction * torch.log(mean_prediction), dim=1)

  for j in range(0,len(res)):
      c=i*b+j
      kernel_distace_SVHN[c] = res[j].item()
  if(i%20==0):
    print(c)


fig = plt.figure(figsize=(5,3))
er=[0]

for p,e in enumerate(er):
    sample=(test_dataset_CIFAR[10][0]+e*m*torch.randn(3,32,32)).numpy()

    kernel_distace_CIFAR_e=np.zeros(r)
    c=0
    for data in ls:
      Data=data+e*m*torch.randn(len(data),3,32,32)
      if mod=='DUQ':
          res=model_CIFAR(Data.cuda())[1].max(1)[0].detach()
      else:
          predictions = torch.stack([model(Data.cuda()) for model in model_CIFAR])
          mean_prediction = torch.mean(predictions.exp(), dim=0)
          res=torch.sum(mean_prediction * torch.log(mean_prediction), dim=1)

      for j in range(0,len(res)):
        kernel_distace_CIFAR_e[c] = res[j].item()
        c+=1
      if(c%1000==0):
        print(c)


    ax=fig.add_subplot(len(er),1,p+1,xlabel='Confidence',ylabel='Freq',title='Hist')
    n, bins, patches = ax.hist(x=[kernel_distace_CIFAR_e,kernel_distace_SVHN], bins='auto', color=['#0000FF','#FFA500']                      )
    ax.legend(['CIFAR','SVHN'],loc='upper left')
    

fig.tight_layout()
plt.show()


Files already downloaded and verified
Files already downloaded and verified
